In [44]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [45]:
df1=pd.read_csv("C:/Users/HP/Downloads/analysis_data.csv/analysis_data.csv")
df1.head()

,locality_name,region_name,price,value_per_sqft,area,construction_status,house_type,total_rooms,total_beds,new_resale,age
0,Tarwala Nagar,Nashik,8499000.0,3761,2260,Under Construction,Apartment,6,4,New,-1.0
1,Dombivali,Mumbai,6790000.0,13580,500,Under Construction,Apartment,4,2,New,0.0
2,Wagholi,Pune,6030000.0,7811,773,Under Construction,Apartment,5,3,New,-3.0
3,Ambegaon Budruk,Pune,3537000.0,7417,477,Ready to move,Apartment,3,1,New,0.5
4,Dombivali,Mumbai,9500000.0,10359,917,Under Construction,Apartment,5,3,New,-3.0


In [46]:
df1 = df1.drop(['locality_name', 'value_per_sqft', 'construction_status', 'house_type','new_resale', 'age','total_beds'],axis='columns')
df1.head()

,region_name,price,area,total_rooms
0,Nashik,8499000.0,2260,6
1,Mumbai,6790000.0,500,4
2,Pune,6030000.0,773,5
3,Pune,3537000.0,477,3
4,Mumbai,9500000.0,917,5


In [47]:
df1.shape

(191274, 4)

In [48]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('region_name'):
        m = np.mean(subdf.price)
        st = np.std(subdf.price)
        reduced_df = subdf[(subdf.price>(m-st)) & (subdf.price<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df2 = remove_pps_outliers(df1)
df2.shape

(180783, 4)

In [49]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('region_name'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('total_rooms'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price),
                'std': np.std(bhk_df.price),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('total_rooms'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df3 = remove_bhk_outliers(df2)
# df8 = df7.copy()
df3.shape

(152480, 4)

In [50]:
df4 = df3.copy()
df4[df4.area/df4.total_rooms<300].head()

,region_name,price,area,total_rooms
0,Ahmednagar,2300000.0,650,3
1,Ahmednagar,5500000.0,1000,4
2,Ahmednagar,5500000.0,1000,4
3,Ahmednagar,5500000.0,1000,4
4,Ahmednagar,3000000.0,560,4


In [51]:
df4.shape

(152480, 4)

In [52]:
df5 = df4[~(df4.area/df4.total_rooms<300)]
df5.shape

(20638, 4)

In [53]:
dummies = pd.get_dummies(df4.region_name)
dummies.head(3)

,Ahmednagar,Akola,Alibaugh,Amravati,Aurangabad,Baramati,Bhandara,Chandrapur,Dhule,Hingoli,...,Pune,Raigad,Ratnagiri,Sangli,Satara,Sindhudurg,Solapur,Vikramgad,Wardha,Yavatmal
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
df5 = pd.concat([df4,dummies],axis='columns')
df5.head()

,region_name,price,area,total_rooms,Ahmednagar,Akola,Alibaugh,Amravati,Aurangabad,Baramati,...,Pune,Raigad,Ratnagiri,Sangli,Satara,Sindhudurg,Solapur,Vikramgad,Wardha,Yavatmal
0,Ahmednagar,2300000.0,650,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ahmednagar,5500000.0,1000,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ahmednagar,5500000.0,1000,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ahmednagar,5500000.0,1000,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ahmednagar,3000000.0,560,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
df5.shape

(152480, 35)

In [56]:
x = df5.drop(['price','region_name'],axis='columns')
x

,area,total_rooms,Ahmednagar,Akola,Alibaugh,Amravati,Aurangabad,Baramati,Bhandara,Chandrapur,...,Pune,Raigad,Ratnagiri,Sangli,Satara,Sindhudurg,Solapur,Vikramgad,Wardha,Yavatmal
0,650,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,560,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180778,1800,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
180779,1800,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
180780,2200,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
180781,1300,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [57]:
y = df5.price
y

0         2300000.0
1         5500000.0
2         5500000.0
3         5500000.0
4         3000000.0
            ...    
180778    8400000.0
180779    8400000.0
180780    4800000.0
180781    7000000.0
180782    7000000.0
Name: price, Length: 152480, dtype: float64

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [59]:
from sklearn.linear_model import LinearRegression
ler = LinearRegression()
ler.fit(X_train,Y_train)
ler.score(X_test,Y_test)

0.6007915218464122

In [73]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), x, y, cv=cv)

array([0.60079152, 0.59286962, 0.59979559, 0.58813085, 0.59340259])

In [75]:
def predict_price(location,sqft,bath):    
    loc_index = np.where(x.columns==location)[0][0]

    pre = np.zeros(len(x.columns))
    pre[0] = sqft
    pre[1] = bath
    if loc_index >= 0:
        x[loc_index] = 1

    return ler.predict([x])[0]

In [147]:
x.head(2)

,area,total_rooms,Ahmednagar,Akola,Alibaugh,Amravati,Aurangabad,Baramati,Bhandara,Chandrapur,...,Pune,Raigad,Ratnagiri,Sangli,Satara,Sindhudurg,Solapur,Vikramgad,Wardha,Yavatmal
0,650,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
ler.predict([[1000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])[0]

7427896.27825984

In [143]:
def predict_price(location,area,rooms):    
    loc_index = np.where(x.columns==location)[0][0]

    pre = np.zeros(len(x.columns))
    pre[0] = area
    pre[1] = rooms
    if loc_index >= 0:
        pre[loc_index] = 1

    return ler.predict([pre])[0]

In [150]:
abs(predict_price(' Akola',650,3))

2828879.584143035

In [145]:
y.head(2)

0    2300000.0
1    5500000.0
Name: price, dtype: float64

In [158]:
import pickle
with open('Pune_home_prices_model.pickle','wb') as f:
    pickle.dump(ler,f)

In [156]:
import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [159]:
import pickle
with open('Pune_home_prices_model.pickle','rb') as f:
    model=pickle.load(f)



In [160]:
def predict_price(location,area,rooms):    
    loc_index = np.where(x.columns==location)[0][0]

    pre = np.zeros(len(x.columns))
    pre[0] = area
    pre[1] = rooms
    if loc_index >= 0:
        pre[loc_index] = 1

    return model.predict([pre])[0]

In [161]:
abs(predict_price(' Akola',650,3))

2828879.584143035